# XGBoost
training a classifier using XGBoost on Dask

In [10]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters


In [11]:
FUNCTION           = 'clf_logreg_dask'               # python function file name
DESCRIPTION        = 'train a LogisticRegression classifier on a Dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'          # custom image
JOB_KIND           = 'job'                                           

TASK_NAME          = 'user-task-train-logreg-dask'     # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'

DASK_CLIENT        = 'scheduler.json'
TRAIN_SET          = ('xtrain', 'ytrain') # published names in the cluster 
VALID_SET          = ('xvalid', 'yvalid')

## load and configure function

In [12]:
FUNC_PY = '/User/repos/demos/dask/code/clf_logreg_dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/clf_logreg_dask.yaml'

In [13]:
clf_logreg = mlrun.new_function(command=FUNC_PY, image=IMAGE, kind=JOB_KIND)
clf_logreg.spec.build.image = IMAGE

clf_logreg.export(FUNC_YAML)
# clf_lgbm = mlrun.import_function(FUNC_YAML)

[mlrun] 2020-02-11 19:29:12,287 function spec saved to path: /User/repos/demos/dask/yaml/clf_logreg_dask.yaml


In [9]:
clf_logreg.apply(mlrun.mount_v3io())

clf_logreg.deploy(skip_deployed=True, with_mlrun=False)

# create and run the task
train_clf_logreg_task = mlrun.NewTask(
    TASK_NAME, 
    handler=FUNCTION,  
    params={
        'dask_client': DASK_CLIENT,
        'train_set'  : TRAIN_SET,
        'valid_set'  : VALID_SET,
        'target_path': ARTIFACTS_PATH,
        'name'       : 'logreg-model.pkl',
        'key'        : 'logreg-model',
        'params'     : {}
    })

# run
rn = clf_logreg.run(train_clf_logreg_task)

[mlrun] 2020-02-11 18:41:09,583 function spec saved to path: /User/repos/demos/dask/yaml/clf_logreg_dask.yaml
[mlrun] 2020-02-11 18:41:09,593 starting run user-task-train-logreg-dask uid=8c577cbe7f5e4c73a7591e06917bbb7c  -> http://mlrun-api:8080
[mlrun] 2020-02-11 18:41:09,783 Job is running in the background, pod: user-task-train-logreg-dask-t645x
[mlrun] 2020-02-11 19:17:19,624 Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/mlrun/runtimes/local.py", line 199, in exec_from_params
    val = handler(*args_list)
  File "/User/repos/demos/dask/clf_logreg_dask.py", line 74, in clf_logreg_dask
  File "/opt/conda/lib/python3.7/site-packages/dask_ml/linear_model/glm.py", line 187, in fit
    self._coef = algorithms._solvers[self.solver](X, y, **solver_kwargs)
  File "/opt/conda/lib/python3.7/site-packages/dask_glm/utils.py", line 26, in normalize_inputs
    out = algo(Xn, y, *args, **kwargs).copy()
  File "/opt/conda/lib/python3.7/site-packages/dask_glm/algo

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...7bbb7c,0,Feb 11 18:41:20,")"">error",clf_logreg_dask,host=user-task-train-logreg-dask-t645xkind=jobowner=admin,,"dask_client=scheduler.jsonkey=logreg-modelname=logreg-model.pklparams={}target_path=/User/repos/demos/dask/artifactstrain_set=['xtrain', 'ytrain']valid_set=['xvalid', 'yvalid']",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8c577cbe7f5e4c73a7591e06917bbb7c  , !mlrun logs 8c577cbe7f5e4c73a7591e06917bbb7c 
[mlrun] 2020-02-11 19:17:24,852 run executed, status=error
runtime error: ("('gt-read-parquet-split-values-5e2c669e76fc54cb5951961472b8cd09', 403)", <Worker 'tcp://10.233.64.53:41797', name: tcp://10.233.64.53:41797, memory: 0, processing: 3969>)


RunError: ("('gt-read-parquet-split-values-5e2c669e76fc54cb5951961472b8cd09', 403)", <Worker 'tcp://10.233.64.53:41797', name: tcp://10.233.64.53:41797, memory: 0, processing: 3969>)